In [1]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import os
import csv

#### Exif functions

In [2]:
#Get Latitude and Longitude from EXIF using PIL
#From https://gist.github.com/erans/983821
def get_exif_data(image):
    """Returns a dictionary from the exif data of an PIL Image item. Also converts the GPS Tags"""
    exif_data = {}
    info = image._getexif()
    if info:
        for tag, value in info.items():
            decoded = TAGS.get(tag, tag)
            if decoded == "GPSInfo":
                gps_data = {}
                for t in value:
                    sub_decoded = GPSTAGS.get(t, t)
                    gps_data[sub_decoded] = value[t]

                exif_data[decoded] = gps_data
            else:
                exif_data[decoded] = value
    return exif_data

def _get_if_exist(data, key):
    """Check if key exist in data, return data if true"""
    if key in data:
        return data[key]
    return None

def _convert_to_degress(value):
    """Helper function to convert the GPS coordinates stored in the EXIF to degress in float format"""
    d0 = value[0][0]
    d1 = value[0][1]
    d = float(d0) / float(d1)

    m0 = value[1][0]
    m1 = value[1][1]
    m = float(m0) / float(m1)

    s0 = value[2][0]
    s1 = value[2][1]
    s = float(s0) / float(s1)
    return d + (m / 60.0) + (s / 3600.0)  
    
def get_lat_lon(exif_data):
    """Get GPS data from exif data"""
#    lat = None
#    lon = None
    if "GPSInfo" in exif_data:
        gps_info = exif_data["GPSInfo"]

        gps_latitude = _get_if_exist(gps_info, "GPSLatitude")
        gps_latitude_ref = _get_if_exist(gps_info, 'GPSLatitudeRef')
        gps_longitude = _get_if_exist(gps_info, 'GPSLongitude')
        gps_longitude_ref = _get_if_exist(gps_info, 'GPSLongitudeRef')

        if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
            lat = _convert_to_degress(gps_latitude)
            if gps_latitude_ref != "N":                     
                lat = 0 - lat
            lon = _convert_to_degress(gps_longitude)
            if gps_longitude_ref != "E":
                lon = 0 - lon
            return lat, lon
        else:
            return None
    else:
            return None

def get_positions_from_folder(img_dir):
    info_list=[]
    for img_name in os.listdir(img_dir):
        img=Image.open(img_dir+img_name)
        exif_data=get_exif_data(img)
        gps_pos=get_lat_lon(exif_data)
        if gps_pos is not None:
            info_list.append([img_name, gps_pos[0], gps_pos[1]])
    #        print(gps_pos)
        else:
            print("No GPS data found! "+img_name)
    return info_list


In [3]:
#Sample usage
image=Image.open("./test1.jpg")
print(image)
sample_data= get_exif_data(image)
gps_pos=get_lat_lon(sample_data)
if gps_pos is not None:
    print(gps_pos)
else:
    print("No GPS data found!")


<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4128x3096 at 0x1CAE0558518>
No GPS data found!


In [4]:
search_dir = "./source_img/"
output_file = "./test.csv"
"""
for img_name in os.listdir(img_dir):
    img=Image.open(img_dir+img_name)
    exif_data=get_exif_data(img)
    gps_pos=get_lat_lon(exif_data)
    if gps_pos is not None:
        info_list.append([img_name, gps_pos[0], gps_pos[1]])
#        print(gps_pos)
    else:
        print("No GPS data found! "+img_name)
"""
listed_data= get_positions_from_folder(search_dir)
if len(listed_data)>1:
    listed_data.insert(0,["File","Lat", "Lon"])

with open(output_file, 'w') as myfile:
    wr = csv.writer(myfile, lineterminator='\n')
    #for i in listed_data:
    wr.writerows(listed_data)
    myfile.close

No GPS data found! DSC_1075.JPG
No GPS data found! DSC_1076.JPG
No GPS data found! DSC_1079.JPG
No GPS data found! DSC_1084.JPG
